## **1. Environment Set-up**

In [1]:
#!pip install pyspark

In [2]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext 
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

groupproject = SparkSession.builder.appName("HM_Recommendations").config("spark.sql.files.maxPartitionBytes", 5000000).getOrCreate()
spark = SparkSession(groupproject)

22/10/19 12:23:09 WARN Utils: Your hostname, Haos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
22/10/19 12:23:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 12:23:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 12:23:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/19 12:23:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## **2. Dataset check & data choose**

In [3]:
transaction = spark.read.option("header",True) \
              .csv("transactions_train.csv")
transaction.printSchema()

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sales_channel_id: string (nullable = true)



In [4]:
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp, lit
min_date, max_date = transaction.select(min("t_dat"), max("t_dat")).first()
print(min_date, max_date)

datahm =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
datahm = datahm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
datahm = datahm.withColumn('year', year(col('date')))
datahm = datahm.withColumn('month', month(col('date')))
datahm = datahm.withColumn('day', date_format(col('date'), "d"))

datahm = datahm[datahm['year'] == 2020]
datahm = datahm[datahm['month'] == 9]
datahm = datahm[datahm['day'] == 22]
transaction.unpersist()

# Prepare the dataset
datahm = datahm.groupby('customer_id', 'article_id').count()
datahm.show()


2018-09-20 2020-09-22


+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|00f7bc5c0df4c615b...|0780418013|    1|
|02094817e46f3b692...|0791587001|    1|
|0333e5dda0257e9f4...|0839332002|    2|
|07c7a1172caf8fb97...|0573085043|    1|
|081373184e601470c...|0714790020|    1|
|09bec2a61046ccbea...|0860336002|    1|
|0be4f1ecce204ee32...|0573085028|    1|
|0c4b30343292b5101...|0918522001|    1|
|0e10e02358875468b...|0579541001|    1|
|0fc371e67e61a31d7...|0907170001|    1|
|10817b19177f6a53e...|0718278019|    1|
|10ac90988da6052dd...|0934212003|    1|
|14a298482fa7f9d52...|0894353002|    1|
|14f4b1b17991c32d2...|0803685001|    1|
|1601fa3c3f39aa623...|0730683001|    1|
|164e1a251f0e3d764...|0831267001|    1|
|165d2c0b0128d5619...|0909081004|    1|
|166546028742fe655...|0767423013|    1|
|189b7275c513a84c8...|0877711001|    1|
|1918933afff08e955...|0914672001|    1|
+--------------------+----------+-----+
only showing top 20 rows



In [5]:
userId_count = datahm.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

+--------------------+-----+
|         customer_id|count|
+--------------------+-----+
|30b6056bacc5f5c9d...|   28|
|5e8fb4d457fdffc61...|   28|
|dc1b173e541f8d3c1...|   27|
|6335d496ef463bc40...|   25|
|1796e87fd2e88932b...|   25|
|f50287d9cf052d4b4...|   24|
|54e8ebd39543b5a4d...|   23|
|fd5ce8716faf00f6a...|   23|
|850ec77661a417d27...|   22|
|ad3663a848dccbdda...|   21|
|32f3a6a7ce63d302c...|   21|
|b606fe5786c00151a...|   21|
|298523b6637340717...|   21|
|b49647f84a99ced53...|   21|
|fc783381f1ea2174c...|   21|
|a08e284bb18add2d7...|   21|
|383e1b07e2c1fe169...|   21|
|3ca77aab50ae4532b...|   20|
|2a721767cd9864ed5...|   20|
|af5166e0f89b0d433...|   19|
+--------------------+-----+
only showing top 20 rows



In [6]:
articleId_count = datahm.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

+----------+-----+
|article_id|count|
+----------+-----+
|0924243002|   91|
|0918522001|   88|
|0866731001|   78|
|0751471001|   75|
|0448509014|   73|
|0714790020|   72|
|0762846027|   68|
|0928206001|   67|
|0893432002|   66|
|0918292001|   65|
|0915529005|   64|
|0788575004|   63|
|0915529003|   63|
|0863583001|   60|
|0930380001|   59|
|0573085028|   59|
|0919273002|   58|
|0850917001|   57|
|0573085042|   56|
|0874110016|   53|
+----------+-----+
only showing top 20 rows



## **3. Model build & Evaluator generate**

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(datahm.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(datahm).transform(datahm)
transformed.show()

22/10/19 12:26:05 WARN DAGScheduler: Broadcasting large task binary with size 1210.8 KiB
+--------------------+----------+-----+----------------+-----------------+
|         customer_id|article_id|count|article_id_index|customer_id_index|
+--------------------+----------+-----+----------------+-----------------+
|00f7bc5c0df4c615b...|0780418013|    1|          2237.0|            783.0|
|02094817e46f3b692...|0791587001|    1|            35.0|            785.0|
|0333e5dda0257e9f4...|0839332002|    2|           732.0|           4098.0|
|07c7a1172caf8fb97...|0573085043|    1|            44.0|           1702.0|
|081373184e601470c...|0714790020|    1|             5.0|           4146.0|
|09bec2a61046ccbea...|0860336002|    1|          2368.0|           6792.0|
|0be4f1ecce204ee32...|0573085028|    1|            14.0|            799.0|
|0c4b30343292b5101...|0918522001|    1|             1.0|           6825.0|
|0e10e02358875468b...|0579541001|    1|            53.0|           2689.0|
|0fc371e67e

In [8]:
#dataset build with randomsplit "8:2"
(training,test)=transformed.randomSplit([0.8, 0.2])

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


#create ALS model
als=ALS(userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)

#tune model using ParamGridBuilder
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [15,20,25])\
            .addGrid(als.maxIter,[5,10,15])\
            .addGrid(als.regParam,[0.09,0.14,0.19])\
            .build()
#define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse",labelCol = 'count', predictionCol = 'prediction')

#Build cross validation using CrossValidator
crossvalidate = CrossValidator(estimator=als,estimatorParamMaps=param_grid, evaluator=evaluator,numFolds=3)

#load the crovalidator into the model
model = crossvalidate.fit(training)


22/10/19 12:26:30 WARN DAGScheduler: Broadcasting large task binary with size 1237.4 KiB


22/10/19 12:26:31 WARN DAGScheduler: Broadcasting large task binary with size 1238.8 KiB


22/10/19 12:26:34 WARN DAGScheduler: Broadcasting large task binary with size 1240.3 KiB


22/10/19 12:26:36 WARN DAGScheduler: Broadcasting large task binary with size 1241.6 KiB


22/10/19 12:26:36 WARN DAGScheduler: Broadcasting large task binary with size 1240.6 KiB


...

22/10/19 12:38:03 WARN DAGScheduler: Broadcasting large task binary with size 1241.7 KiB
22/10/19 12:38:03 WARN DAGScheduler: Broadcasting large task binary with size 1240.7 KiB


22/10/19 12:38:03 WARN DAGScheduler: Broadcasting large task binary with size 1242.0 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1242.7 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1245.8 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1247.2 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1248.6 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1250.0 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1251.4 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1252.8 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1254.2 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1255.5 KiB
22/10/19 12:38:04 WARN DAGScheduler: Broadcasting large task binary with size 1256.9 KiB
22/10/19 12:38:04 WAR

22/10/19 12:38:09 WARN DAGScheduler: Broadcasting large task binary with size 1355.9 KiB


22/10/19 12:39:04 WARN DAGScheduler: Broadcasting large task binary with size 1227.4 KiB


22/10/19 12:39:04 WARN DAGScheduler: Broadcasting large task binary with size 1228.8 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1230.3 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1231.6 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1230.6 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1231.9 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1232.6 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/19 12:39:05 WARN DAGScheduler: Broadcasting large task binary with size 1238.5 KiB
22/10/19 12:39:06 WARN DAGScheduler: Broadcasting large task binary with size 1239.9 KiB
22/10/19 12:39:06 WARN DAGScheduler: Broadcasting large task binary with size 1241.3 KiB
22/10/19 12:39:06 WAR

In [10]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

22/10/19 12:39:36 WARN DAGScheduler: Broadcasting large task binary with size 1227.4 KiB


22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1228.8 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1230.3 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1231.6 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1230.6 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1231.9 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1232.6 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1235.7 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1237.1 KiB
22/10/19 12:39:37 WARN DAGScheduler: Broadcasting large task binary with size 1238.5 KiB
22/10/19 12:39:38 WARN DAGScheduler: Broadcasting large task binary with size 1239.9 KiB
22/10/19 12:39:38 WARN DAGScheduler: Broadcasting large task binary with size 1241.3 KiB
22/10/19 12:39:38 WAR

## **4.Prediction & Evaluate**

In [11]:
#Generate predictions and evaluate using RMSE
predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)

22/10/19 12:39:39 WARN DAGScheduler: Broadcasting large task binary with size 1256.4 KiB


22/10/19 12:39:40 WARN DAGScheduler: Broadcasting large task binary with size 1255.1 KiB


22/10/19 12:40:09 WARN DAGScheduler: Broadcasting large task binary with size 1308.5 KiB


In [12]:
#print evaluation metrics and model parameters
print("RMSE =" + str(rmse))

RMSE =0.47437555346437627
